In [63]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from vincenty import vincenty

from sklearn.preprocessing import scale
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
from sklearn.datasets import fetch_mldata
from sklearn.utils import shuffle
import pickle

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 10000)

In [131]:
#Import natural disaster data
raw = pd.read_csv('natural_disaster_human_mobility.csv')

In [170]:
#Switch latitude and longtitude columns, rename and reorder, dataframe
df = raw
df = df.rename(columns = {'disaster.event': 'event', 'user.anon': 'user', 'latitude': 'long', 'longitude.anon': 'lat'})
df = df[['event', 'user', 'lat', 'long', 'time']]
df['time'] = pd.DatetimeIndex(df['time'])


In [171]:
#Clean event names
event_name={
'01_Wipha':'Wipha',
'02_Halong': 'Halong',
'06_Kalmaegi': 'Kalmaegi',
'08_Rammasun_Manila': 'Manila',
'12_Bohol': 'Bohol',
'13_Iquique': 'Iquidue',
'14_Napa': 'Napa',
'21_Norfolk': 'Norfolk',
'22_Hamburg': 'Hamburg',
'23_Atlanta': 'Atlanta',
'31_Phoenix': 'Phoenix',
'32_Detroit': 'Detroit',
'33_Baltimore': 'Baltimore',
'41_AuFire1': 'Australia1',
'42_AuFire2': 'Australia2'}

df['event_name'] = df['event'].map(event_name)

In [172]:
#Add event type
event_type={
'Wipha': 'Typhoon',
'Halong': 'Typhoon',
'Kalmaegi': 'Typhoon',
'Manila': 'Typhoon',
'Bohol': 'Earthquake',
'Iquidue': 'Earthquake',
'Napa': 'Earthquake',
'Norfolk': 'Winter_Storm',
'Hamburg': 'Winter_Storm',
'Atlanta': 'Winter_Storm',
'Phoenix': 'Thunderstorm',
'Detroit': 'Thunderstorm',
'Baltimore': 'Thunderstorm',
'Australia1': 'Wildfire',
'Australia2': 'Wildfire'}    

df['event_type'] = df['event_name'].map(event_type)
df = df.drop(columns = ['event'])

In [173]:
#Remove Wipia, has bad data 300,000 enteries!
df = df.loc[df['event_name'] != 'Wipha']

In [174]:
#Add date column, drop time column
df['date'] = df['time'].dt.date
df['date'] = pd.to_datetime(df['date'])

In [73]:
#Disaster day function and key
def disaster_day(event_name, date):
    if event_name == 'Wipha' and str(date) == '2013-10-11 00:00:00':
        return True
    elif event_name == 'Wipha' and str(date) == '2013-10-12 00:00:00':
        return True
    elif event_name == 'Wipha' and str(date) == '2013-10-13 00:00:00':
        return True
    elif event_name == 'Wipha' and str(date) == '2013-10-14 00:00:00':
        return True
    elif event_name == 'Wipha' and str(date) == '2013-10-15 00:00:00':
        return True    
    elif event_name == 'Wipha' and str(date) == '2013-10-16 00:00:00':
        return True    
    elif event_name == 'Manila' and str(date) == '2014-07-11 00:00:00':
        return True    
    elif event_name == 'Manila' and str(date) == '2014-07-12 00:00:00':
        return True  
    elif event_name == 'Manila' and str(date) == '2014-07-13 00:00:00':
        return True  
    elif event_name == 'Manila' and str(date) == '2014-07-14 00:00:00':
        return True  
    elif event_name == 'Manila' and str(date) == '2014-07-15 00:00:00':
        return True      
    elif event_name == 'Manila' and str(date) == '2014-07-16 00:00:00':
        return True    
    elif event_name == 'Manila' and str(date) == '2014-07-17 00:00:00':
        return True  
    elif event_name == 'Manila' and str(date) == '2014-07-18 00:00:00':
        return True  
    elif event_name == 'Bohol' and str(date) == '2013-10-15 00:00:00':
        return True 
    elif event_name == 'Iquidue' and str(date) == '2014-01-18 00:00:00':
        return True     
    elif event_name == 'Napa' and str(date) == '2014-08-24 00:00:00':
        return True   
    elif event_name == 'Atlanta' and str(date) == '2014-01-08 00:00:00':
        return True   
    elif event_name == 'Detroit' and str(date) == '2014-09-20 00:00:00':
        return True   
    elif event_name == 'Australia1' and str(date) == '2013-10-17 00:00:00':
        return True 
    elif event_name == 'Australia1' and str(date) == '2013-10-18 00:00:00':
        return True     
    elif event_name == 'Australia1' and str(date) == '2013-10-19 00:00:00':
        return True     
    elif event_name == 'Australia1' and str(date) == '2013-10-20 00:00:00':
        return True     
    elif event_name == 'Australia1' and str(date) == '2013-10-21 00:00:00':
        return True  
    elif event_name == 'Australia1' and str(date) == '2013-10-22 00:00:00':
        return True     
    elif event_name == 'Australia1' and str(date) == '2013-10-23 00:00:00':
        return True     
    elif event_name == 'Australia1' and str(date) == '2013-10-24 00:00:00':
        return True     
    elif event_name == 'Australia1' and str(date) == '2013-10-25 00:00:00':
        return True     
    elif event_name == 'Australia1' and str(date) == '2013-10-26 00:00:00':
        return True 
    elif event_name == 'Australia1' and str(date) == '2013-10-27 00:00:00':
        return True     
    elif event_name == 'Australia1' and str(date) == '2013-10-28 00:00:00':
        return True     
    else:
        return False

In [ ]:
#Apply disaster day fucntion
#df_small['disaster_day'] = df_small.apply(lambda row: disaster_day((row['event_name']), (row['date'])), axis = 1)

In [200]:
df_small =df.iloc[:200,]

In [202]:
#Sort data by time
df_small = df_small.groupby(['event_name', 'user'], group_keys=False)
df_small = df_small.apply(lambda _df_small: _df_small.sort_values(by=['time']))

In [203]:
#Generate distance between tweets column
df_small['lat_1'] = df_small['lat'].shift()
df_small['long_1'] = df_small['long'].shift()
df_small['dist_mi'] = df_small.apply(lambda row: vincenty((row['lat'], row['long']), (row['lat_1'], row['long_1'])), axis=1)

In [205]:
#Delete first row in every groupby
df_small = df_small.groupby(['event_name', 'user']).apply(lambda row: row.iloc[1:])

In [208]:
#Create tweet velocity column
df_small['time_diff'] = df_small['time'].diff()
df_small['time_diff_hour'] = df_small['time_diff']/np.timedelta64(1, 'h')
df_small['time_diff_day'] = df_small['time_diff']/np.timedelta64(1, 'D')
df_small['velocity_mph'] = df_small['dist_mi']/df_small['time_diff_hour']
df_small['dist_cent'] = df_small.apply(lambda row: vincenty((row['lat'], row['long']), \
                                                    (df_small['lat'].mean(), df_small['long'].mean())), axis=1)

In [209]:
df_small.head(50)

user        lat        long                time  \
event_name user                                                            
Halong     2    1416908     2  43.178472  127.823767 2014-08-05 23:56:06   
                1416909     2  43.178472  127.823767 2014-08-05 23:56:37   
                1416910     2  43.178472  127.823767 2014-08-05 23:57:26   
                1416911     2  43.178472  127.823767 2014-08-05 23:58:07   
                1416912     2  43.178472  127.823767 2014-08-05 23:59:00   
                1416913     2  43.178472  127.823767 2014-08-06 00:00:29   
                1416914     2  43.178472  127.823767 2014-08-06 00:01:22   
                1416915     2  43.178472  127.823767 2014-08-06 00:02:27   
                1416916     2  43.178472  127.823767 2014-08-06 00:03:22   
                1416917     2  43.178472  127.823767 2014-08-06 00:03:58   
                1416918     2  43.178472  127.823767 2014-08-06 00:05:04   
                1416919     2  43.178472  127.823767 2014-08-06 00:06:23   
                1416920     2  43.178472  127.823767 2014-08-06 00:07:12   
                1416921     2  43.178472  127.823767 2014-08-06 00:07:57   
                1416922     2  43.178472  127.823767 2014-08-06 00:08:57   
                1416923     2  43.178472  127.823767 2014-08-06 00:09:25   
                1416924     2  43.178472  127.823767 2014-08-06 00:10:53   
                1416925     2  43.178472  127.823767 2014-08-06 06:40:19   
                1416926     2  43.178472  127.823767 2014-08-06 06:43:13   
                1416927     2  43.178472  127.823767 2014-08-06 06:44:41   
                1416928     2  43.178472  127.823767 2014-08-06 06:46:19   
                1416929     2  43.178472  127.823767 2014-08-06 06:47:07   
                1416930     2  43.178472  127.823767 2014-08-06 06:48:20   
                1416931     2  43.178472  127.823767 2014-08-06 06:49:22   
                1416932     2  43.178472  127.823767 2014-08-06 09:36:36   
                1416933     2  43.178472  127.823767 2014-08-06 09:37:31   
                1416934     2  43.178472  127.823767 2014-08-06 09:38:21   
                1416935     2  43.178472  127.823767 2014-08-06 09:38:40   
                1416936     2  43.175704  127.818604 2014-08-06 11:46:15   
                1416937     2  42.934874  127.699940 2014-08-06 17:15:04   
                1416938     2  42.934874  127.699940 2014-08-06 17:19:03   
                1416939     2  42.934874  127.699940 2014-08-06 17:20:17   
                1416940     2  42.934874  127.699940 2014-08-06 17:23:04   
                1416941     2  42.934874  127.699940 2014-08-06 17:23:44   
                1416942     2  42.934874  127.699940 2014-08-06 17:25:58   
                1416943     2  42.934874  127.699940 2014-08-06 17:27:01   
                1416944     2  42.928652  127.683989 2014-08-06 22:02:12   
                1416945     2  42.928652  127.683989 2014-08-06 22:03:35   
                1416946     2  42.928652  127.683989 2014-08-06 23:00:47   
           3    1416948     3  42.920647  127.697782 2014-08-23 17:08:36   
           4    1416949     4  43.043096  127.769233 2014-08-01 10:49:47   
                1416950     4  43.312659  127.969970 2014-08-02 08:35:44   
                1416951     4  43.313300  127.967870 2014-08-04 13:00:59   
                1416952     4  43.308270  127.968322 2014-08-18 16:35:19   
           5    1416955     5  43.071679  127.829222 2014-07-22 18:36:39   
                1416956     5  43.070407  127.835288 2014-07-26 17:44:11   
                1416957     5  43.070455  127.835242 2014-07-26 22:33:37   
           7    1416960     7  42.928083  127.688344 2014-08-19 07:53:06   
           8    1416978     8  42.987546  127.730999 2014-07-24 10:22:28   
                1416980     8  42.986507  127.729840 2014-07-25 17:41:44   

                        event_name event_type       date      lat_1  \
e

In [113]:
#Generate distance column
#all_data['lat_1'] = all_data['lat'].shift()
#all_data['long_1'] = all_data['long'].shift()
#all_data['dist_mi'] = all_data.apply(lambda row: vincenty((row['lat'], row['long']), (row['lat_1'], row['long_1'])), axis=1)

In [ ]:
all_data['date'] = all_data['time'].dt.date
all_data['time_diff'] = all_data['time'].diff()
all_data['time_diff_hour'] = all_data['time_diff']/np.timedelta64(1, 'h')
all_data['time_diff_day'] = all_data['time_diff']/np.timedelta64(1, 'D')
all_data['velocity_mph'] = all_data['dist_mi']/all_data['time_diff_hour']
all_data['dist_cent'] = napa.apply(lambda row: vincenty((row['lat'], row['long']), \
                                                    (all_data['lat'].mean(), all_data['long'].mean())), axis=1)
                                                    

In [ ]:
#Calculate event location by averaging the first lat long entry for each user
all_data.groupby('event').max()

In [ ]:
#Export average coordinates to csv for tableau
group.to_csv('ave_coord.csv')

In [ ]:
napa2 = df[(df['event'] == '14_Napa')]
napa2.shape
napa2.to_csv('napa2.csv')
napa2.head()

In [50]:
df.head()

,user,lat,long,time,event_name,event_type,date,disaster_day
0,0,24.515364,139.742561,2013-10-20 10:46:03,Wipha,Typhoon,2013-10-20,False
1,0,24.569996,139.702880,2013-10-20 18:04:00,Wipha,Typhoon,2013-10-20,False
2,0,24.550507,139.701482,2013-10-20 18:04:47,Wipha,Typhoon,2013-10-20,False
3,0,24.508685,139.728029,2013-10-20 18:14:42,Wipha,Typhoon,2013-10-20,False
4,0,24.576521,139.702315,2013-10-04 20:16:17,Wipha,Typhoon,2013-10-04,False


In [53]:
napa = df[(df['event_name'] == 'Napa')]
napa = napa.iloc[0:1000,:]
napa.head(100)

,user,lat,long,time,event_name,event_type,date,disaster_day


In [58]:
df2 = df.groupby(['event_name']).mean()
df2

,user,lat,long,disaster_day
event_name,,,,
Atlanta,7803.335969,48.115655,-84.402464,0.023076
Australia1,2572.768018,-22.569533,151.059366,0.334674
Australia2,3991.742691,-31.948435,151.008412,0.000000
Baltimore,7527.589023,21.977349,-76.620763,0.000000
Bohol,3805.313771,-2.370050,123.860556,0.051324
Detroit,8281.965728,38.071186,-83.117919,0.000000
Halong,2482.137684,43.006657,127.762166,0.000000
Hamburg,1428.430413,63.611212,10.002712,0.000000
Kalmaegi,591.625357,5.342795,120.410101,0.000000


In [ ]:
napa['time_diff'] = napa['time'].diff()
napa['time_diff_hour'] = napa['time_diff']/np.timedelta64(1, 'h')
napa['time_diff_day'] = napa['time_diff']/np.timedelta64(1, 'D')

In [ ]:
napa['velocity_mph'] = napa['dist_mi']/napa['time_diff_hour'] 

In [ ]:
napa['dist_cent'] = napa.apply(lambda row: vincenty((row['lat'], row['long']), \
                                                    (napa['lat'].mean(), napa['long'].mean())), axis=1)


In [ ]:
napa.to_csv('napa.csv')

In [ ]:
napa['date'] = napa['time'].dt.date
napa['date'] = pd.to_datetime(napa['date'])

In [ ]:
#napa['disaster_day'] = napa.where(napa['date'] == '2014-08-24')
napa['disaster_day'] = napa['date'].map(lambda x: 1 if x== '2014-08-24' else 0)

In [ ]:
napa['disaster_day'] = (napa['date'] == '2014-08-24')
#sol['Completed'] = (sol['Installed Status'] == 'Installed').astype(int) 

In [ ]:
napa.head(300)

In [ ]:
napa.info()

In [ ]:
napa_group_user = napa.groupby(['user', 'date', 'disaster_day']).mean()
napa_group_user

In [ ]:

#pd.to_datetime(napa['time'],format = '%m/%d/%Y')

In [ ]:
napa_group = napa.groupby(by = 'date').mean().reset_index()
napa_group

In [ ]:
napa_group.info()

In [ ]:
napa_group3 = napa_group.reset_index()
napa_group3

In [ ]:
napa_group2 = napa.groupby(by = 'date').size()
napa_group2

In [ ]:
sns.scatterplot(x=napa_group2, y=napa_group['velocity_mph'])

In [ ]:
sns.scatterplot(x=napa_group3['lat'], y=napa_group3['long'], hue=napa_group3['date'])

In [ ]:
sns.scatterplot(x=napa_group3['lat'], y=napa_group3['long'])

In [ ]:
X = np.append(napa_group3['lat'],napa_group3['long']).transpose()

In [ ]:
X = napa_group3[['lat', 'long']]

In [ ]:
X = X.as_matrix()

In [ ]:
X.shape

In [ ]:
#helper function that allows us to display data in 2 dimensions an highlights the clusters
def display_cluster(X,km=[],num_clusters=0):
    color = 'brgcmyk'
    alpha = 0.5
    s = 20
    if num_clusters == 0:
        plt.scatter(X[:,0],X[:,1],c = color[0],alpha = alpha,s = s)
    else:
        for i in range(num_clusters):
            plt.scatter(X[km.labels_==i,0],X[km.labels_==i,1],c = color[i],alpha = alpha,s=s)
            plt.scatter(km.cluster_centers_[i][0],km.cluster_centers_[i][1],c = color[i], marker = 'x', s = 100)

In [ ]:
num_clusters = 2
km = KMeans(n_clusters=num_clusters,random_state=10,n_init=1) # n_init, number of times the K-mean algorithm will run
km.fit(X)
display_cluster(X,km,num_clusters)

In [ ]:
km.cluster_centers_

In [ ]:
km.inertia_

In [ ]:
plt.plot([KMeans(n_clusters=i).fit(X).inertia_ for i in range(1,10)])

In [ ]:
df_small = df_small.groupby(['event_name', 'user']).apply(lambda _df_small: _df_small.sort_values(by=['time']))

In [ ]:
#Pickle data
redo_pickle = False
if redo_pickle == True:
    with open('df.pickle','wb') as f:
        pickle.dump(df,f)
#if redo_pickle == False:
    #with open('df.pickle','rb') as f:
        #df = pickle.load(f)

In [ ]:
#Add natural disaster tag
'Wipha' and '2013-10-11', '2013-10-12', '2013-10-13', '2013-10-14', '2013-10-15', '2013-10-16'
#'Halong'
#'Kalmaegi'
'Manila' and '2014-7-11', '2014-7-12', '2014-7-13', '2014-7-14', '2014-7-15', '2014-7-16', '2014-7-17', '2014-7-18'
'Bohol' and '2013-10-15'
'Iquidue'and '2014-04-01'
'Napa' and '2014-08-24
'Norfolk'
'Hamburg'
'Atlanta' '2014-01-28'
'Phoenix': 
'Detroit': '2014-09-20'
'Baltimore': 
'Australia1': 2013-10-17, 2013-10-28
'Australia2': 

    